In [1]:
# -*- coding: utf-8 -*-
import warnings
warnings.filterwarnings('ignore')

import csv, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.externals import joblib
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

pd.set_option('display.max_columns', 500)
curr_dir = '/home/souche/qiongjiu/hgc/'

In [2]:
data_train = pd.read_csv(curr_dir+"2020-11-02版采购数据data_train2.csv", header=0)
data_test = pd.read_csv(curr_dir+"2020-11-02版采购数据data_test2.csv", header=0)
print("按级别划分前训练数据数目为:%d" % data_train.shape[0])
print("按级别划分前训练数据包含的车型数为:%d" % len(data_train['model_code'].unique()))
print("按级别划分前测试数据数目为:%d" % data_test.shape[0])
print("按级别划分前测试数据包含的车型数为:%d" % len(data_test['model_code'].unique()))

按级别划分前训练数据数目为:1117766
按级别划分前训练数据包含的车型数为:25303
按级别划分前测试数据数目为:281881
按级别划分前测试数据包含的车型数为:22381


In [3]:
'keep_value' in data_train.columns

True

In [4]:
car = pd.read_csv(curr_dir+"2020-11-02版车型参数及独热编码.csv", header = 0, low_memory=False)
car=car.drop(['model_year','new_car_price', 'driving_mode','gear_box_type','country_id',
 'import_type','intake_type', 'fuel_form','quality_mile', 'quality_year',
 'year_1','year_2','year_3','year_4','year_5','year_6','year_7','year_8',
 'year_9', 'year_10','year_11','year_12','year_13','year_14','year_15',
 'year_16', 'car_body','rate','rate_count'],axis=1)

data_train = pd.merge(data_train, car, how="inner", on="model_code")
data_test = pd.merge(data_test, car, how="inner", on="model_code")
print(data_train.shape)
print(data_test.shape)

(1117766, 211)
(281881, 211)


In [5]:
'keep_value' in data_train.columns

True

In [6]:
vec = pd.read_csv(curr_dir+"2020-11-02版-TF-IDF权重的词向量.csv", dtype={'model_code': str}, header = 0)
vec = vec.drop(['model_name'], axis=1)
print(vec.shape)

data_train = pd.merge(data_train, vec, how="inner", on="model_code")
data_test = pd.merge(data_test, vec, how="inner", on="model_code")
print(data_train.shape)
print(data_test.shape)

(65250, 101)
(1117766, 311)
(281881, 311)


### 先计算采购B级车误差中位数

In [7]:
train_B = data_train[(data_train['level']=="B") | 
                     (data_train['level']=="C") | 
                     (data_train['level']=="D") | 
                     (data_train['level']=="E")]
print("筛选出B级以上的车训练集数量为:%d" % train_B.shape[0])
print("筛选出B级以上的车训练集--车型数量为:%d" % len(train_B['model_code'].unique()))
print("<<-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*>>")

test_B = data_test[(data_test['level']=="B") | 
                   (data_test['level']=="C") | 
                   (data_test['level']=="D") | 
                   (data_test['level']=="E")]
print("筛选出B级以上的车测试集数量为:%d" % test_B.shape[0])
print("筛选出B级以上的车测试集--车型数量为:%d" % len(test_B['model_code'].unique()))

筛选出B级以上的车训练集数量为:534014
筛选出B级以上的车训练集--车型数量为:9851
<<-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*>>
筛选出B级以上的车测试集数量为:134420
筛选出B级以上的车测试集--车型数量为:8900


In [8]:
# 从train_A中再抽取10%的数据作为validation set
val_B = train_B.sample(frac=0.1, random_state=0)
train_B = train_B.drop(val_B.index)
print(val_B.shape, train_B.shape)

print(train_B['level'].unique())
print(val_B['level'].unique())
print(test_B['level'].unique())

(53401, 311) (480613, 311)
['B' 'C' 'D' 'E']
['C' 'B' 'D' 'E']
['B' 'C' 'D' 'E']


In [9]:
# 将训练集测试集打乱
train_B = shuffle(train_B)
val_B = shuffle(val_B)
test_B = shuffle(test_B)

train_X_B = train_B.drop(['car_id', 'brand_code', 'brand_name', 'series_code',
                      'series_name', 'model_code', 'model_name',  'color_name',
                      'first_license_plate_date', 'province_name', 'city_name',
                      'purchase_price', 'order_create_time', 'residual', 'mileage_std', 'license_month',
                      'new_car_price', 'count', 'series_level', 'level', 'level0', 'level1',
                      'level2', 'level3', 'level4', 'level5', 'level6', 'level7', 'level8'], axis=1)
train_y_B = train_B['residual']

val_X_B = val_B.drop(['car_id', 'brand_code', 'brand_name', 'series_code',
                      'series_name', 'model_code', 'model_name',  'color_name',
                      'first_license_plate_date', 'province_name', 'city_name',
                      'purchase_price', 'order_create_time', 'residual', 'mileage_std', 'license_month',
                      'new_car_price', 'count', 'series_level', 'level', 'level0', 'level1',
                      'level2', 'level3', 'level4', 'level5', 'level6', 'level7', 'level8'], axis=1)
val_y_B = val_B['residual']

test_X_B = test_B.drop(['car_id', 'brand_code', 'brand_name', 'series_code',
                      'series_name', 'model_code', 'model_name',  'color_name',
                      'first_license_plate_date', 'province_name', 'city_name',
                      'purchase_price', 'order_create_time', 'residual', 'mileage_std', 'license_month',
                      'new_car_price', 'count', 'series_level', 'level', 'level0', 'level1',
                      'level2', 'level3', 'level4', 'level5', 'level6', 'level7', 'level8'], axis=1)
test_y_B = test_B['residual']

In [10]:
print(train_X_B.shape)
print(test_X_B.shape)
print(val_X_B.shape)

(480613, 282)
(134420, 282)
(53401, 282)


In [11]:
import lightgbm as lgb

gbm = lgb.LGBMRegressor(boosting_type = 'gbdt', 
                        objective = 'regression', 
                        learning_rate = 0.04, 
                        n_estimators = 2800, 
                        max_depth = 8, 
                        num_leaves = 150, 
                        subsample = 0.95, 
                        colsample_bytree = 0.3, 
                        min_child_samples = 100, 
                        n_jobs = 8, 
                        verbose = 1)

gbm.fit(train_X_B, 
        train_y_B,
        eval_set = (val_X_B, val_y_B),
        eval_metric = {'l1','l2'}, 
        early_stopping_rounds = 50, 
        verbose=True)

[1]	valid_0's l2: 0.0369716	valid_0's l1: 0.158251
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's l2: 0.0357935	valid_0's l1: 0.15569
[3]	valid_0's l2: 0.0334054	valid_0's l1: 0.150307
[4]	valid_0's l2: 0.031239	valid_0's l1: 0.145287
[5]	valid_0's l2: 0.0292425	valid_0's l1: 0.140488
[6]	valid_0's l2: 0.0275149	valid_0's l1: 0.136145
[7]	valid_0's l2: 0.0257876	valid_0's l1: 0.131706
[8]	valid_0's l2: 0.0242032	valid_0's l1: 0.127504
[9]	valid_0's l2: 0.022744	valid_0's l1: 0.123506
[10]	valid_0's l2: 0.0218861	valid_0's l1: 0.121118
[11]	valid_0's l2: 0.0210431	valid_0's l1: 0.118725
[12]	valid_0's l2: 0.0202519	valid_0's l1: 0.116446
[13]	valid_0's l2: 0.0195443	valid_0's l1: 0.114341
[14]	valid_0's l2: 0.0184153	valid_0's l1: 0.110918
[15]	valid_0's l2: 0.0173757	valid_0's l1: 0.107639
[16]	valid_0's l2: 0.0164023	valid_0's l1: 0.104506
[17]	valid_0's l2: 0.0158674	valid_0's l1: 0.102717
[18]	valid_0's l2: 0.015054	valid_0's l1: 0.0999609
[19]	valid_0's

[154]	valid_0's l2: 0.00289271	valid_0's l1: 0.0404122
[155]	valid_0's l2: 0.00288953	valid_0's l1: 0.0403823
[156]	valid_0's l2: 0.00288252	valid_0's l1: 0.0403216
[157]	valid_0's l2: 0.00287955	valid_0's l1: 0.0402968
[158]	valid_0's l2: 0.0028729	valid_0's l1: 0.0402371
[159]	valid_0's l2: 0.00287081	valid_0's l1: 0.0402181
[160]	valid_0's l2: 0.00286762	valid_0's l1: 0.0401918
[161]	valid_0's l2: 0.00286346	valid_0's l1: 0.040157
[162]	valid_0's l2: 0.00286024	valid_0's l1: 0.0401282
[163]	valid_0's l2: 0.00285639	valid_0's l1: 0.0400938
[164]	valid_0's l2: 0.00285495	valid_0's l1: 0.0400805
[165]	valid_0's l2: 0.00285203	valid_0's l1: 0.0400551
[166]	valid_0's l2: 0.00285037	valid_0's l1: 0.0400395
[167]	valid_0's l2: 0.00284463	valid_0's l1: 0.0399889
[168]	valid_0's l2: 0.00283904	valid_0's l1: 0.039943
[169]	valid_0's l2: 0.00283757	valid_0's l1: 0.0399293
[170]	valid_0's l2: 0.00283279	valid_0's l1: 0.0398907
[171]	valid_0's l2: 0.00282765	valid_0's l1: 0.0398492
[172]	valid_0

[304]	valid_0's l2: 0.00252332	valid_0's l1: 0.0371736
[305]	valid_0's l2: 0.00252304	valid_0's l1: 0.0371706
[306]	valid_0's l2: 0.00252024	valid_0's l1: 0.037143
[307]	valid_0's l2: 0.00251964	valid_0's l1: 0.0371373
[308]	valid_0's l2: 0.00251693	valid_0's l1: 0.0371118
[309]	valid_0's l2: 0.00251566	valid_0's l1: 0.0371011
[310]	valid_0's l2: 0.00251398	valid_0's l1: 0.0370856
[311]	valid_0's l2: 0.00251292	valid_0's l1: 0.0370767
[312]	valid_0's l2: 0.00251091	valid_0's l1: 0.0370618
[313]	valid_0's l2: 0.00250893	valid_0's l1: 0.0370472
[314]	valid_0's l2: 0.00250846	valid_0's l1: 0.0370427
[315]	valid_0's l2: 0.00250699	valid_0's l1: 0.0370288
[316]	valid_0's l2: 0.00250513	valid_0's l1: 0.0370129
[317]	valid_0's l2: 0.00250433	valid_0's l1: 0.0370057
[318]	valid_0's l2: 0.00250368	valid_0's l1: 0.0369996
[319]	valid_0's l2: 0.00250321	valid_0's l1: 0.0369943
[320]	valid_0's l2: 0.00250257	valid_0's l1: 0.0369887
[321]	valid_0's l2: 0.00250177	valid_0's l1: 0.0369815
[322]	valid

[457]	valid_0's l2: 0.00237006	valid_0's l1: 0.0357683
[458]	valid_0's l2: 0.00236895	valid_0's l1: 0.0357587
[459]	valid_0's l2: 0.00236866	valid_0's l1: 0.0357568
[460]	valid_0's l2: 0.00236821	valid_0's l1: 0.0357529
[461]	valid_0's l2: 0.00236774	valid_0's l1: 0.0357485
[462]	valid_0's l2: 0.00236697	valid_0's l1: 0.0357425
[463]	valid_0's l2: 0.00236628	valid_0's l1: 0.0357361
[464]	valid_0's l2: 0.00236502	valid_0's l1: 0.0357251
[465]	valid_0's l2: 0.00236393	valid_0's l1: 0.0357153
[466]	valid_0's l2: 0.00236356	valid_0's l1: 0.0357124
[467]	valid_0's l2: 0.00236272	valid_0's l1: 0.0357042
[468]	valid_0's l2: 0.00236203	valid_0's l1: 0.0356982
[469]	valid_0's l2: 0.00236124	valid_0's l1: 0.0356903
[470]	valid_0's l2: 0.00236081	valid_0's l1: 0.0356862
[471]	valid_0's l2: 0.00236064	valid_0's l1: 0.0356846
[472]	valid_0's l2: 0.0023599	valid_0's l1: 0.0356776
[473]	valid_0's l2: 0.00235921	valid_0's l1: 0.0356718
[474]	valid_0's l2: 0.00235862	valid_0's l1: 0.0356663
[475]	valid

[609]	valid_0's l2: 0.00228218	valid_0's l1: 0.0349435
[610]	valid_0's l2: 0.00228114	valid_0's l1: 0.0349336
[611]	valid_0's l2: 0.00228094	valid_0's l1: 0.034932
[612]	valid_0's l2: 0.00228074	valid_0's l1: 0.0349305
[613]	valid_0's l2: 0.00227991	valid_0's l1: 0.034922
[614]	valid_0's l2: 0.00227969	valid_0's l1: 0.0349201
[615]	valid_0's l2: 0.00227921	valid_0's l1: 0.0349156
[616]	valid_0's l2: 0.00227877	valid_0's l1: 0.0349111
[617]	valid_0's l2: 0.00227825	valid_0's l1: 0.0349062
[618]	valid_0's l2: 0.00227729	valid_0's l1: 0.0348974
[619]	valid_0's l2: 0.0022769	valid_0's l1: 0.034894
[620]	valid_0's l2: 0.00227623	valid_0's l1: 0.0348885
[621]	valid_0's l2: 0.00227556	valid_0's l1: 0.0348819
[622]	valid_0's l2: 0.002275	valid_0's l1: 0.0348771
[623]	valid_0's l2: 0.00227445	valid_0's l1: 0.034872
[624]	valid_0's l2: 0.00227415	valid_0's l1: 0.0348688
[625]	valid_0's l2: 0.00227354	valid_0's l1: 0.0348633
[626]	valid_0's l2: 0.00227332	valid_0's l1: 0.0348617
[627]	valid_0's l

...

[2273]	valid_0's l2: 0.00199224	valid_0's l1: 0.0321218
[2274]	valid_0's l2: 0.00199219	valid_0's l1: 0.0321208
[2275]	valid_0's l2: 0.00199197	valid_0's l1: 0.0321183
[2276]	valid_0's l2: 0.00199194	valid_0's l1: 0.0321178
[2277]	valid_0's l2: 0.00199193	valid_0's l1: 0.0321175
[2278]	valid_0's l2: 0.00199189	valid_0's l1: 0.0321173
[2279]	valid_0's l2: 0.00199186	valid_0's l1: 0.032117
[2280]	valid_0's l2: 0.00199182	valid_0's l1: 0.0321167
[2281]	valid_0's l2: 0.00199169	valid_0's l1: 0.0321156
[2282]	valid_0's l2: 0.00199162	valid_0's l1: 0.032115
[2283]	valid_0's l2: 0.00199148	valid_0's l1: 0.0321134
[2284]	valid_0's l2: 0.00199132	valid_0's l1: 0.0321118
[2285]	valid_0's l2: 0.00199117	valid_0's l1: 0.0321107
[2286]	valid_0's l2: 0.00199106	valid_0's l1: 0.0321093
[2287]	valid_0's l2: 0.00199102	valid_0's l1: 0.0321089
[2288]	valid_0's l2: 0.00199082	valid_0's l1: 0.0321069
[2289]	valid_0's l2: 0.00199081	valid_0's l1: 0.0321067
[2290]	valid_0's l2: 0.00199069	valid_0's l1: 0.03

[2421]	valid_0's l2: 0.00198009	valid_0's l1: 0.0320004
[2422]	valid_0's l2: 0.00198002	valid_0's l1: 0.0319996
[2423]	valid_0's l2: 0.00197998	valid_0's l1: 0.0319992
[2424]	valid_0's l2: 0.00197979	valid_0's l1: 0.0319965
[2425]	valid_0's l2: 0.00197969	valid_0's l1: 0.0319957
[2426]	valid_0's l2: 0.00197955	valid_0's l1: 0.0319943
[2427]	valid_0's l2: 0.00197952	valid_0's l1: 0.0319937
[2428]	valid_0's l2: 0.00197932	valid_0's l1: 0.0319918
[2429]	valid_0's l2: 0.00197924	valid_0's l1: 0.0319908
[2430]	valid_0's l2: 0.00197916	valid_0's l1: 0.0319899
[2431]	valid_0's l2: 0.00197898	valid_0's l1: 0.0319882
[2432]	valid_0's l2: 0.00197893	valid_0's l1: 0.0319876
[2433]	valid_0's l2: 0.00197885	valid_0's l1: 0.031987
[2434]	valid_0's l2: 0.00197876	valid_0's l1: 0.031986
[2435]	valid_0's l2: 0.00197857	valid_0's l1: 0.0319848
[2436]	valid_0's l2: 0.00197844	valid_0's l1: 0.0319832
[2437]	valid_0's l2: 0.00197839	valid_0's l1: 0.0319828
[2438]	valid_0's l2: 0.00197824	valid_0's l1: 0.03

[2569]	valid_0's l2: 0.00196945	valid_0's l1: 0.0318962
[2570]	valid_0's l2: 0.00196942	valid_0's l1: 0.0318963
[2571]	valid_0's l2: 0.00196936	valid_0's l1: 0.0318956
[2572]	valid_0's l2: 0.00196929	valid_0's l1: 0.0318948
[2573]	valid_0's l2: 0.00196925	valid_0's l1: 0.0318943
[2574]	valid_0's l2: 0.00196917	valid_0's l1: 0.0318938
[2575]	valid_0's l2: 0.00196908	valid_0's l1: 0.0318929
[2576]	valid_0's l2: 0.00196904	valid_0's l1: 0.0318927
[2577]	valid_0's l2: 0.00196899	valid_0's l1: 0.0318921
[2578]	valid_0's l2: 0.00196893	valid_0's l1: 0.0318916
[2579]	valid_0's l2: 0.00196882	valid_0's l1: 0.0318906
[2580]	valid_0's l2: 0.00196874	valid_0's l1: 0.03189
[2581]	valid_0's l2: 0.0019686	valid_0's l1: 0.0318889
[2582]	valid_0's l2: 0.00196846	valid_0's l1: 0.0318879
[2583]	valid_0's l2: 0.00196841	valid_0's l1: 0.0318872
[2584]	valid_0's l2: 0.0019683	valid_0's l1: 0.0318861
[2585]	valid_0's l2: 0.00196823	valid_0's l1: 0.0318852
[2586]	valid_0's l2: 0.00196818	valid_0's l1: 0.0318

[2719]	valid_0's l2: 0.00195817	valid_0's l1: 0.031793
[2720]	valid_0's l2: 0.00195812	valid_0's l1: 0.0317926
[2721]	valid_0's l2: 0.00195808	valid_0's l1: 0.0317921
[2722]	valid_0's l2: 0.00195799	valid_0's l1: 0.0317914
[2723]	valid_0's l2: 0.00195786	valid_0's l1: 0.0317902
[2724]	valid_0's l2: 0.0019578	valid_0's l1: 0.0317894
[2725]	valid_0's l2: 0.00195778	valid_0's l1: 0.0317894
[2726]	valid_0's l2: 0.00195763	valid_0's l1: 0.0317877
[2727]	valid_0's l2: 0.00195756	valid_0's l1: 0.0317874
[2728]	valid_0's l2: 0.00195752	valid_0's l1: 0.0317869
[2729]	valid_0's l2: 0.0019575	valid_0's l1: 0.0317866
[2730]	valid_0's l2: 0.00195736	valid_0's l1: 0.0317856
[2731]	valid_0's l2: 0.00195727	valid_0's l1: 0.031785
[2732]	valid_0's l2: 0.00195719	valid_0's l1: 0.0317844
[2733]	valid_0's l2: 0.0019571	valid_0's l1: 0.0317838
[2734]	valid_0's l2: 0.00195697	valid_0's l1: 0.0317824
[2735]	valid_0's l2: 0.00195692	valid_0's l1: 0.0317819
[2736]	valid_0's l2: 0.00195688	valid_0's l1: 0.03178

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.3,
       learning_rate=0.04, max_depth=8, min_child_samples=100,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=2800,
       n_jobs=8, num_leaves=150, objective='regression', random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=0.95,
       subsample_for_bin=200000, subsample_freq=0, verbose=1)

In [13]:
# 测试集误差打印
def print_test_result(ypred, test_y):
    err3, err5, err8, err10 = 0, 0, 0, 0
    test_y = np.array(test_y)
    size = len(ypred)
    for i in range(size):
        err = abs(ypred[i] - test_y[i])/test_y[i]
        if(err<=0.03):
            err3 +=1
        if(err<=0.05):
            err5 += 1
        if(err<=0.08):
            err8 +=1
        if(err<=0.1):
            err10 += 1
    print('预测误差在百分之 3以内数目为：%5d,所占比例为：%.4f%%'% (err3, err3/size*100))
    print('预测误差在百分之 5以内数目为：%5d,所占比例为：%.4f%%'% (err5, err5/size*100))
    print('预测误差在百分之 8以内数目为：%5d,所占比例为：%.4f%%'% (err8, err8/size*100))
    print('预测误差在百分之10以内数目为：%5d,所占比例为：%.4f%%'% (err10, err10/size*100))

In [14]:
ypred = gbm.predict(test_X_B)
print_test_result(ypred, test_y_B)

预测误差在百分之 3以内数目为：44767,所占比例为：33.3038%
预测误差在百分之 5以内数目为：67866,所占比例为：50.4880%
预测误差在百分之 8以内数目为：92156,所占比例为：68.5583%
预测误差在百分之10以内数目为：103049,所占比例为：76.6620%


In [15]:
by_model = test_B[['model_code', 'count', 'residual']]
by_model['pre_residual'] = ypred
by_model['pre_err'] = abs(by_model['residual'] - by_model['pre_residual'])
by_model['percent_err'] = by_model['pre_err']/by_model['residual']

data_result = pd.DataFrame(columns={'model_code':str, 'median_res':float})
data_result['model_code'] = list(by_model.groupby(['model_code'])['percent_err'].quantile(0.5).index)
data_result['median_res'] = list(by_model.groupby(['model_code'])['percent_err'].quantile(0.5))

print(data_result[data_result['median_res']<=0.03].shape[0], 
      data_result[data_result['median_res']<=0.03].shape[0]/data_result.shape[0])
print(data_result[data_result['median_res']<=0.05].shape[0], 
      data_result[data_result['median_res']<=0.05].shape[0]/data_result.shape[0])
print(data_result[data_result['median_res']<=0.08].shape[0], 
      data_result[data_result['median_res']<=0.08].shape[0]/data_result.shape[0])
print(data_result[data_result['median_res']<=0.1].shape[0], 
      data_result[data_result['median_res']<=0.1].shape[0]/data_result.shape[0])

1656 0.18606741573033708
3708 0.4166292134831461
5862 0.6586516853932585
6671 0.7495505617977528


In [16]:
data_result = data_result[['model_code', 'median_res']]
data_result.to_csv(curr_dir+"2020-11-02采购数据-B级车-误差中位数.csv", header = True, index=False)

### 计算采购A级车误差中位数

In [18]:
train_A = data_train[(data_train['level']=="缺失") | 
                     (data_train['level']=="A00") | 
                     (data_train['level']=="A0") | 
                     (data_train['level']=="A")]
print("筛选出A级以上的车训练集数量为:%d" % train_A.shape[0])
print("筛选出A级以上的车训练集--车型数量为:%d" % len(train_A['model_code'].unique()))
print("<<-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*>>")

test_A = data_test[(data_test['level']=="缺失") | 
                   (data_test['level']=="A00") | 
                   (data_test['level']=="A0") | 
                   (data_test['level']=="A")]
print("筛选出A级以上的车测试集数量为:%d" % test_A.shape[0])
print("筛选出A级以上的车测试集--车型数量为:%d" % len(test_A['model_code'].unique()))

筛选出A级以上的车训练集数量为:583499
筛选出A级以上的车训练集--车型数量为:15417
<<-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*>>
筛选出A级以上的车测试集数量为:147392
筛选出A级以上的车测试集--车型数量为:13455


In [19]:
# 从train_A中再抽取10%的数据作为validation set
val_A = train_A.sample(frac=0.1, random_state=0)
train_A = train_A.drop(val_A.index)
print(val_A.shape, train_A.shape)

print(train_A['level'].unique())
print(val_A['level'].unique())
print(test_A['level'].unique())

(58350, 311) (525149, 311)
['A' 'A0' 'A00' '缺失']
['A0' 'A' '缺失' 'A00']
['A' 'A0' '缺失' 'A00']


In [20]:
#将训练集测试集打乱
train_A = shuffle(train_A)
val_A = shuffle(val_A)
test_A = shuffle(test_A)

train_X_A = train_A.drop(['car_id', 'brand_code', 'brand_name', 'series_code',
                      'series_name', 'model_code', 'model_name',  'color_name',
                      'first_license_plate_date', 'province_name', 'city_name',
                      'purchase_price', 'order_create_time', 'residual', 'mileage_std', 'license_month',
                      'new_car_price', 'count', 'series_level', 'level', 'level0', 'level1',
                      'level2', 'level3', 'level4', 'level5', 'level6', 'level7', 'level8'], axis=1)
train_y_A = train_A['residual']

val_X_A = val_A.drop(['car_id', 'brand_code', 'brand_name', 'series_code',
                      'series_name', 'model_code', 'model_name',  'color_name',
                      'first_license_plate_date', 'province_name', 'city_name',
                      'purchase_price', 'order_create_time', 'residual', 'mileage_std', 'license_month',
                      'new_car_price', 'count', 'series_level', 'level', 'level0', 'level1',
                      'level2', 'level3', 'level4', 'level5', 'level6', 'level7', 'level8'], axis=1)
val_y_A = val_A['residual']

test_X_A = test_A.drop(['car_id', 'brand_code', 'brand_name', 'series_code',
                      'series_name', 'model_code', 'model_name',  'color_name',
                      'first_license_plate_date', 'province_name', 'city_name',
                      'purchase_price', 'order_create_time', 'residual', 'mileage_std', 'license_month',
                      'new_car_price', 'count', 'series_level', 'level', 'level0', 'level1',
                      'level2', 'level3', 'level4', 'level5', 'level6', 'level7', 'level8'], axis=1)
test_y_A = test_A['residual']

print(train_X_A.shape, train_y_A.shape)
print(val_X_A.shape, val_y_A.shape)
print(test_X_A.shape, test_y_A.shape)

(525149, 282) (525149,)
(58350, 282) (58350,)
(147392, 282) (147392,)


In [23]:
import lightgbm as lgb
gbm = lgb.LGBMRegressor(boosting_type = 'gbdt', 
                        objective = 'regression', 
                        learning_rate = 0.025, 
                        n_estimators = 2800, 
                        max_depth = 9, 
                        num_leaves = 120, 
                        subsample = 0.95, 
                        colsample_bytree = 0.4, 
                        min_child_samples = 110, 
                        verbose = 1, 
                        n_jobs=8)

gbm.fit(train_X_A, 
        train_y_A,
        eval_set = (val_X_A, val_y_A),
        eval_metric = {'l1','l2'}, 
        early_stopping_rounds = 50, 
        verbose=True)


[1]	valid_0's l2: 0.0364214	valid_0's l1: 0.157903
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's l2: 0.0352788	valid_0's l1: 0.155353
[3]	valid_0's l2: 0.0339286	valid_0's l1: 0.152257
[4]	valid_0's l2: 0.0325936	valid_0's l1: 0.149158
[5]	valid_0's l2: 0.0315567	valid_0's l1: 0.146711
[6]	valid_0's l2: 0.0302906	valid_0's l1: 0.143678
[7]	valid_0's l2: 0.0291946	valid_0's l1: 0.140967
[8]	valid_0's l2: 0.0280209	valid_0's l1: 0.13803
[9]	valid_0's l2: 0.027162	valid_0's l1: 0.135849
[10]	valid_0's l2: 0.0260996	valid_0's l1: 0.133097
[11]	valid_0's l2: 0.0251292	valid_0's l1: 0.130539
[12]	valid_0's l2: 0.0245542	valid_0's l1: 0.128977
[13]	valid_0's l2: 0.0236176	valid_0's l1: 0.126436
[14]	valid_0's l2: 0.0227459	valid_0's l1: 0.124028
[15]	valid_0's l2: 0.0218918	valid_0's l1: 0.121622
[16]	valid_0's l2: 0.021101	valid_0's l1: 0.119349
[17]	valid_0's l2: 0.0203352	valid_0's l1: 0.117103
[18]	valid_0's l2: 0.0195966	valid_0's l1: 0.114906
[19]	valid_0's

[153]	valid_0's l2: 0.00399474	valid_0's l1: 0.0490543
[154]	valid_0's l2: 0.0039895	valid_0's l1: 0.0490118
[155]	valid_0's l2: 0.00398335	valid_0's l1: 0.0489598
[156]	valid_0's l2: 0.00397698	valid_0's l1: 0.0489072
[157]	valid_0's l2: 0.00397223	valid_0's l1: 0.048868
[158]	valid_0's l2: 0.00396395	valid_0's l1: 0.0488039
[159]	valid_0's l2: 0.00395781	valid_0's l1: 0.0487545
[160]	valid_0's l2: 0.00395319	valid_0's l1: 0.0487171
[161]	valid_0's l2: 0.00394851	valid_0's l1: 0.0486789
[162]	valid_0's l2: 0.00394338	valid_0's l1: 0.0486369
[163]	valid_0's l2: 0.00393549	valid_0's l1: 0.0485761
[164]	valid_0's l2: 0.0039275	valid_0's l1: 0.0485129
[165]	valid_0's l2: 0.00392241	valid_0's l1: 0.0484707
[166]	valid_0's l2: 0.00391697	valid_0's l1: 0.0484244
[167]	valid_0's l2: 0.00390979	valid_0's l1: 0.0483705
[168]	valid_0's l2: 0.00390193	valid_0's l1: 0.0483089
[169]	valid_0's l2: 0.00389542	valid_0's l1: 0.0482624
[170]	valid_0's l2: 0.0038914	valid_0's l1: 0.0482294
[171]	valid_0'

[303]	valid_0's l2: 0.00347619	valid_0's l1: 0.0449117
[304]	valid_0's l2: 0.00347421	valid_0's l1: 0.0448964
[305]	valid_0's l2: 0.00347271	valid_0's l1: 0.044883
[306]	valid_0's l2: 0.00347173	valid_0's l1: 0.0448736
[307]	valid_0's l2: 0.00346976	valid_0's l1: 0.0448557
[308]	valid_0's l2: 0.00346854	valid_0's l1: 0.0448451
[309]	valid_0's l2: 0.00346671	valid_0's l1: 0.0448311
[310]	valid_0's l2: 0.00346473	valid_0's l1: 0.0448149
[311]	valid_0's l2: 0.00346249	valid_0's l1: 0.0447973
[312]	valid_0's l2: 0.00345967	valid_0's l1: 0.0447764
[313]	valid_0's l2: 0.00345741	valid_0's l1: 0.0447583
[314]	valid_0's l2: 0.00345513	valid_0's l1: 0.0447405
[315]	valid_0's l2: 0.00345265	valid_0's l1: 0.0447202
[316]	valid_0's l2: 0.00345071	valid_0's l1: 0.0447052
[317]	valid_0's l2: 0.00344947	valid_0's l1: 0.0446945
[318]	valid_0's l2: 0.00344753	valid_0's l1: 0.0446794
[319]	valid_0's l2: 0.00344557	valid_0's l1: 0.0446631
[320]	valid_0's l2: 0.00344426	valid_0's l1: 0.0446505
[321]	valid

[453]	valid_0's l2: 0.00326504	valid_0's l1: 0.0432094
[454]	valid_0's l2: 0.00326405	valid_0's l1: 0.0432016
[455]	valid_0's l2: 0.00326269	valid_0's l1: 0.0431917
[456]	valid_0's l2: 0.00326207	valid_0's l1: 0.0431863
[457]	valid_0's l2: 0.00326124	valid_0's l1: 0.0431794
[458]	valid_0's l2: 0.00326024	valid_0's l1: 0.0431712
[459]	valid_0's l2: 0.00325903	valid_0's l1: 0.0431612
[460]	valid_0's l2: 0.0032573	valid_0's l1: 0.0431472
[461]	valid_0's l2: 0.00325578	valid_0's l1: 0.0431359
[462]	valid_0's l2: 0.00325482	valid_0's l1: 0.0431284
[463]	valid_0's l2: 0.00325389	valid_0's l1: 0.043121
[464]	valid_0's l2: 0.00325258	valid_0's l1: 0.0431115
[465]	valid_0's l2: 0.00325195	valid_0's l1: 0.0431059
[466]	valid_0's l2: 0.00325048	valid_0's l1: 0.0430937
[467]	valid_0's l2: 0.00324929	valid_0's l1: 0.0430843
[468]	valid_0's l2: 0.00324841	valid_0's l1: 0.0430758
[469]	valid_0's l2: 0.00324717	valid_0's l1: 0.0430667
[470]	valid_0's l2: 0.00324628	valid_0's l1: 0.0430603
[471]	valid_

[604]	valid_0's l2: 0.00314099	valid_0's l1: 0.0422038
[605]	valid_0's l2: 0.00314055	valid_0's l1: 0.0422005
[606]	valid_0's l2: 0.00314018	valid_0's l1: 0.0421979
[607]	valid_0's l2: 0.00313927	valid_0's l1: 0.0421905
[608]	valid_0's l2: 0.00313818	valid_0's l1: 0.0421826
[609]	valid_0's l2: 0.00313794	valid_0's l1: 0.0421803
[610]	valid_0's l2: 0.00313706	valid_0's l1: 0.0421731
[611]	valid_0's l2: 0.00313671	valid_0's l1: 0.0421705
[612]	valid_0's l2: 0.00313632	valid_0's l1: 0.0421676
[613]	valid_0's l2: 0.00313546	valid_0's l1: 0.0421609
[614]	valid_0's l2: 0.00313441	valid_0's l1: 0.0421528
[615]	valid_0's l2: 0.00313421	valid_0's l1: 0.0421507
[616]	valid_0's l2: 0.00313354	valid_0's l1: 0.0421452
[617]	valid_0's l2: 0.0031328	valid_0's l1: 0.0421389
[618]	valid_0's l2: 0.00313194	valid_0's l1: 0.0421316
[619]	valid_0's l2: 0.00313149	valid_0's l1: 0.0421274
[620]	valid_0's l2: 0.0031309	valid_0's l1: 0.0421229
[621]	valid_0's l2: 0.00313043	valid_0's l1: 0.0421184
[622]	valid_

...

[2250]	valid_0's l2: 0.00275501	valid_0's l1: 0.0390366
[2251]	valid_0's l2: 0.0027549	valid_0's l1: 0.039036
[2252]	valid_0's l2: 0.00275483	valid_0's l1: 0.0390355
[2253]	valid_0's l2: 0.00275473	valid_0's l1: 0.0390347
[2254]	valid_0's l2: 0.0027545	valid_0's l1: 0.0390328
[2255]	valid_0's l2: 0.00275441	valid_0's l1: 0.039032
[2256]	valid_0's l2: 0.00275426	valid_0's l1: 0.0390307
[2257]	valid_0's l2: 0.00275414	valid_0's l1: 0.0390296
[2258]	valid_0's l2: 0.00275406	valid_0's l1: 0.039029
[2259]	valid_0's l2: 0.00275401	valid_0's l1: 0.0390285
[2260]	valid_0's l2: 0.00275391	valid_0's l1: 0.0390278
[2261]	valid_0's l2: 0.00275369	valid_0's l1: 0.0390262
[2262]	valid_0's l2: 0.00275361	valid_0's l1: 0.0390256
[2263]	valid_0's l2: 0.00275348	valid_0's l1: 0.0390246
[2264]	valid_0's l2: 0.00275338	valid_0's l1: 0.0390235
[2265]	valid_0's l2: 0.00275329	valid_0's l1: 0.0390229
[2266]	valid_0's l2: 0.00275322	valid_0's l1: 0.0390224
[2267]	valid_0's l2: 0.00275307	valid_0's l1: 0.03902

[2398]	valid_0's l2: 0.00273895	valid_0's l1: 0.0389073
[2399]	valid_0's l2: 0.0027388	valid_0's l1: 0.0389059
[2400]	valid_0's l2: 0.00273875	valid_0's l1: 0.0389055
[2401]	valid_0's l2: 0.00273862	valid_0's l1: 0.0389043
[2402]	valid_0's l2: 0.00273855	valid_0's l1: 0.0389038
[2403]	valid_0's l2: 0.00273843	valid_0's l1: 0.0389027
[2404]	valid_0's l2: 0.00273836	valid_0's l1: 0.0389022
[2405]	valid_0's l2: 0.0027382	valid_0's l1: 0.0389014
[2406]	valid_0's l2: 0.0027381	valid_0's l1: 0.0389008
[2407]	valid_0's l2: 0.00273801	valid_0's l1: 0.0389001
[2408]	valid_0's l2: 0.00273791	valid_0's l1: 0.0388993
[2409]	valid_0's l2: 0.00273782	valid_0's l1: 0.0388985
[2410]	valid_0's l2: 0.00273772	valid_0's l1: 0.0388976
[2411]	valid_0's l2: 0.00273768	valid_0's l1: 0.038897
[2412]	valid_0's l2: 0.00273758	valid_0's l1: 0.0388964
[2413]	valid_0's l2: 0.00273747	valid_0's l1: 0.0388954
[2414]	valid_0's l2: 0.00273731	valid_0's l1: 0.0388943
[2415]	valid_0's l2: 0.00273725	valid_0's l1: 0.0388

[2547]	valid_0's l2: 0.00272215	valid_0's l1: 0.0387731
[2548]	valid_0's l2: 0.00272201	valid_0's l1: 0.0387723
[2549]	valid_0's l2: 0.00272192	valid_0's l1: 0.0387717
[2550]	valid_0's l2: 0.00272186	valid_0's l1: 0.038771
[2551]	valid_0's l2: 0.00272178	valid_0's l1: 0.0387703
[2552]	valid_0's l2: 0.00272159	valid_0's l1: 0.0387689
[2553]	valid_0's l2: 0.00272153	valid_0's l1: 0.0387683
[2554]	valid_0's l2: 0.0027214	valid_0's l1: 0.0387671
[2555]	valid_0's l2: 0.00272133	valid_0's l1: 0.0387662
[2556]	valid_0's l2: 0.0027213	valid_0's l1: 0.0387657
[2557]	valid_0's l2: 0.00272119	valid_0's l1: 0.038765
[2558]	valid_0's l2: 0.00272103	valid_0's l1: 0.0387634
[2559]	valid_0's l2: 0.00272097	valid_0's l1: 0.0387628
[2560]	valid_0's l2: 0.00272086	valid_0's l1: 0.0387621
[2561]	valid_0's l2: 0.00272079	valid_0's l1: 0.0387615
[2562]	valid_0's l2: 0.00272061	valid_0's l1: 0.03876
[2563]	valid_0's l2: 0.00272045	valid_0's l1: 0.038759
[2564]	valid_0's l2: 0.00272042	valid_0's l1: 0.0387586

[2696]	valid_0's l2: 0.00270789	valid_0's l1: 0.0386582
[2697]	valid_0's l2: 0.00270772	valid_0's l1: 0.0386569
[2698]	valid_0's l2: 0.00270771	valid_0's l1: 0.0386568
[2699]	valid_0's l2: 0.00270761	valid_0's l1: 0.0386561
[2700]	valid_0's l2: 0.00270752	valid_0's l1: 0.0386552
[2701]	valid_0's l2: 0.00270741	valid_0's l1: 0.038654
[2702]	valid_0's l2: 0.00270733	valid_0's l1: 0.0386534
[2703]	valid_0's l2: 0.00270719	valid_0's l1: 0.0386522
[2704]	valid_0's l2: 0.00270703	valid_0's l1: 0.0386511
[2705]	valid_0's l2: 0.00270694	valid_0's l1: 0.0386503
[2706]	valid_0's l2: 0.00270686	valid_0's l1: 0.0386497
[2707]	valid_0's l2: 0.00270674	valid_0's l1: 0.0386489
[2708]	valid_0's l2: 0.00270661	valid_0's l1: 0.0386477
[2709]	valid_0's l2: 0.00270649	valid_0's l1: 0.0386467
[2710]	valid_0's l2: 0.00270639	valid_0's l1: 0.0386458
[2711]	valid_0's l2: 0.00270634	valid_0's l1: 0.0386455
[2712]	valid_0's l2: 0.0027063	valid_0's l1: 0.038645
[2713]	valid_0's l2: 0.00270617	valid_0's l1: 0.038

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.4,
       learning_rate=0.025, max_depth=9, min_child_samples=110,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=2800,
       n_jobs=8, num_leaves=120, objective='regression', random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=0.95,
       subsample_for_bin=200000, subsample_freq=0, verbose=1)

In [24]:
train_A['pred'] = gbm.predict(train_X_A)
train_A['err'] = abs(train_A['pred'] - train_A['residual'])
train_A[train_A['err']>=0.1].shape[0]

22871

In [26]:
train_A_new = train_A[train_A['err']<0.1]

train_X_A = train_A_new.drop(['car_id', 'brand_code', 'brand_name', 'series_code',
                      'series_name', 'model_code', 'model_name',  'color_name',
                      'first_license_plate_date', 'province_name', 'city_name',
                      'purchase_price', 'order_create_time', 'residual', 'mileage_std', 'license_month',
                      'new_car_price', 'count', 'series_level', 'level', 'level0', 'level1',
                      'level2', 'level3', 'level4', 'level5', 'level6', 'level7', 'level8','pred','err'], axis=1)
train_y_A = train_A_new['residual']

In [29]:
print(train_X_A.shape, train_y_A.shape)
print(val_X_A.shape, val_y_A.shape)
print(test_X_A.shape, test_y_A.shape)

(502278, 282) (502278,)
(58350, 282) (58350,)
(147392, 282) (147392,)


In [30]:
gbm = lgb.LGBMRegressor(boosting_type = 'gbdt', 
                        objective = 'regression', 
                        learning_rate = 0.025, 
                        n_estimators = 2800, 
                        max_depth = 9, 
                        num_leaves = 120, 
                        subsample = 0.95, 
                        colsample_bytree = 0.4, 
                        min_child_samples = 110, 
                        verbose = 1, 
                        n_jobs=8)

gbm.fit(train_X_A, 
        train_y_A,
        eval_set = (val_X_A, val_y_A),
        eval_metric = {'l1','l2'}, 
        early_stopping_rounds = 50, 
        verbose=True)


[1]	valid_0's l2: 0.0364148	valid_0's l1: 0.157887
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's l2: 0.0349341	valid_0's l1: 0.154577
[3]	valid_0's l2: 0.0335874	valid_0's l1: 0.151487
[4]	valid_0's l2: 0.0322236	valid_0's l1: 0.148308
[5]	valid_0's l2: 0.0311136	valid_0's l1: 0.145687
[6]	valid_0's l2: 0.030359	valid_0's l1: 0.143854
[7]	valid_0's l2: 0.029692	valid_0's l1: 0.1422
[8]	valid_0's l2: 0.0286749	valid_0's l1: 0.139684
[9]	valid_0's l2: 0.0278201	valid_0's l1: 0.137536
[10]	valid_0's l2: 0.0267379	valid_0's l1: 0.134766
[11]	valid_0's l2: 0.0257066	valid_0's l1: 0.132084
[12]	valid_0's l2: 0.0247329	valid_0's l1: 0.129509
[13]	valid_0's l2: 0.0242264	valid_0's l1: 0.128137
[14]	valid_0's l2: 0.0236459	valid_0's l1: 0.12653
[15]	valid_0's l2: 0.0229285	valid_0's l1: 0.124542
[16]	valid_0's l2: 0.0220658	valid_0's l1: 0.122108
[17]	valid_0's l2: 0.0212561	valid_0's l1: 0.119779
[18]	valid_0's l2: 0.0204635	valid_0's l1: 0.117459
[19]	valid_0's l

[155]	valid_0's l2: 0.00394822	valid_0's l1: 0.048152
[156]	valid_0's l2: 0.00394294	valid_0's l1: 0.0481082
[157]	valid_0's l2: 0.00393606	valid_0's l1: 0.0480525
[158]	valid_0's l2: 0.00392831	valid_0's l1: 0.0479937
[159]	valid_0's l2: 0.00392184	valid_0's l1: 0.0479408
[160]	valid_0's l2: 0.00391716	valid_0's l1: 0.0479017
[161]	valid_0's l2: 0.00390971	valid_0's l1: 0.0478408
[162]	valid_0's l2: 0.0039024	valid_0's l1: 0.0477798
[163]	valid_0's l2: 0.00389774	valid_0's l1: 0.0477398
[164]	valid_0's l2: 0.00389185	valid_0's l1: 0.0476917
[165]	valid_0's l2: 0.00388493	valid_0's l1: 0.0476389
[166]	valid_0's l2: 0.00387841	valid_0's l1: 0.0475856
[167]	valid_0's l2: 0.00387432	valid_0's l1: 0.0475497
[168]	valid_0's l2: 0.00386671	valid_0's l1: 0.0474838
[169]	valid_0's l2: 0.00385993	valid_0's l1: 0.0474234
[170]	valid_0's l2: 0.00385417	valid_0's l1: 0.0473763
[171]	valid_0's l2: 0.00385042	valid_0's l1: 0.0473451
[172]	valid_0's l2: 0.00384562	valid_0's l1: 0.0472982
[173]	valid_

[307]	valid_0's l2: 0.00349813	valid_0's l1: 0.0442478
[308]	valid_0's l2: 0.00349567	valid_0's l1: 0.0442312
[309]	valid_0's l2: 0.00349357	valid_0's l1: 0.044215
[310]	valid_0's l2: 0.00349302	valid_0's l1: 0.0442097
[311]	valid_0's l2: 0.00349082	valid_0's l1: 0.0441935
[312]	valid_0's l2: 0.00348863	valid_0's l1: 0.0441755
[313]	valid_0's l2: 0.00348605	valid_0's l1: 0.0441548
[314]	valid_0's l2: 0.00348347	valid_0's l1: 0.0441362
[315]	valid_0's l2: 0.00348201	valid_0's l1: 0.0441229
[316]	valid_0's l2: 0.00348111	valid_0's l1: 0.0441142
[317]	valid_0's l2: 0.00347906	valid_0's l1: 0.0440984
[318]	valid_0's l2: 0.00347702	valid_0's l1: 0.044083
[319]	valid_0's l2: 0.00347455	valid_0's l1: 0.0440628
[320]	valid_0's l2: 0.00347273	valid_0's l1: 0.0440471
[321]	valid_0's l2: 0.00347126	valid_0's l1: 0.0440355
[322]	valid_0's l2: 0.00346987	valid_0's l1: 0.0440238
[323]	valid_0's l2: 0.00346848	valid_0's l1: 0.0440126
[324]	valid_0's l2: 0.00346644	valid_0's l1: 0.0439963
[325]	valid_

[458]	valid_0's l2: 0.00330644	valid_0's l1: 0.0426603
[459]	valid_0's l2: 0.00330544	valid_0's l1: 0.0426529
[460]	valid_0's l2: 0.00330454	valid_0's l1: 0.0426449
[461]	valid_0's l2: 0.00330404	valid_0's l1: 0.0426411
[462]	valid_0's l2: 0.00330298	valid_0's l1: 0.0426321
[463]	valid_0's l2: 0.00330184	valid_0's l1: 0.0426218
[464]	valid_0's l2: 0.00330069	valid_0's l1: 0.0426124
[465]	valid_0's l2: 0.00329984	valid_0's l1: 0.0426057
[466]	valid_0's l2: 0.00329913	valid_0's l1: 0.0425994
[467]	valid_0's l2: 0.00329804	valid_0's l1: 0.042589
[468]	valid_0's l2: 0.00329666	valid_0's l1: 0.0425782
[469]	valid_0's l2: 0.00329584	valid_0's l1: 0.0425708
[470]	valid_0's l2: 0.00329475	valid_0's l1: 0.0425609
[471]	valid_0's l2: 0.00329389	valid_0's l1: 0.0425542
[472]	valid_0's l2: 0.00329301	valid_0's l1: 0.0425471
[473]	valid_0's l2: 0.00329182	valid_0's l1: 0.0425366
[474]	valid_0's l2: 0.00329114	valid_0's l1: 0.042531
[475]	valid_0's l2: 0.00329078	valid_0's l1: 0.042528
[476]	valid_0

[608]	valid_0's l2: 0.00320334	valid_0's l1: 0.0417601
[609]	valid_0's l2: 0.00320288	valid_0's l1: 0.0417565
[610]	valid_0's l2: 0.00320262	valid_0's l1: 0.0417542
[611]	valid_0's l2: 0.00320193	valid_0's l1: 0.0417485
[612]	valid_0's l2: 0.00320153	valid_0's l1: 0.0417442
[613]	valid_0's l2: 0.00320113	valid_0's l1: 0.0417401
[614]	valid_0's l2: 0.00320049	valid_0's l1: 0.0417348
[615]	valid_0's l2: 0.00320019	valid_0's l1: 0.0417318
[616]	valid_0's l2: 0.00319998	valid_0's l1: 0.0417295
[617]	valid_0's l2: 0.00319975	valid_0's l1: 0.0417271
[618]	valid_0's l2: 0.00319947	valid_0's l1: 0.0417249
[619]	valid_0's l2: 0.00319877	valid_0's l1: 0.0417191
[620]	valid_0's l2: 0.00319825	valid_0's l1: 0.041714
[621]	valid_0's l2: 0.00319751	valid_0's l1: 0.0417087
[622]	valid_0's l2: 0.00319694	valid_0's l1: 0.0417037
[623]	valid_0's l2: 0.00319664	valid_0's l1: 0.0417013
[624]	valid_0's l2: 0.00319604	valid_0's l1: 0.041696
[625]	valid_0's l2: 0.00319577	valid_0's l1: 0.0416925
[626]	valid_

...

[2248]	valid_0's l2: 0.00289138	valid_0's l1: 0.0387991
[2249]	valid_0's l2: 0.00289124	valid_0's l1: 0.0387981
[2250]	valid_0's l2: 0.00289118	valid_0's l1: 0.0387974
[2251]	valid_0's l2: 0.00289115	valid_0's l1: 0.0387969
[2252]	valid_0's l2: 0.00289108	valid_0's l1: 0.0387962
[2253]	valid_0's l2: 0.00289103	valid_0's l1: 0.0387956
[2254]	valid_0's l2: 0.00289094	valid_0's l1: 0.0387947
[2255]	valid_0's l2: 0.00289088	valid_0's l1: 0.0387941
[2256]	valid_0's l2: 0.00289081	valid_0's l1: 0.0387935
[2257]	valid_0's l2: 0.0028907	valid_0's l1: 0.0387922
[2258]	valid_0's l2: 0.00289067	valid_0's l1: 0.0387915
[2259]	valid_0's l2: 0.00289057	valid_0's l1: 0.0387906
[2260]	valid_0's l2: 0.00289054	valid_0's l1: 0.0387901
[2261]	valid_0's l2: 0.00289039	valid_0's l1: 0.0387889
[2262]	valid_0's l2: 0.00289033	valid_0's l1: 0.0387882
[2263]	valid_0's l2: 0.00289029	valid_0's l1: 0.0387877
[2264]	valid_0's l2: 0.00289021	valid_0's l1: 0.038787
[2265]	valid_0's l2: 0.00289011	valid_0's l1: 0.03

[2396]	valid_0's l2: 0.00288062	valid_0's l1: 0.0386861
[2397]	valid_0's l2: 0.00288055	valid_0's l1: 0.0386853
[2398]	valid_0's l2: 0.00288044	valid_0's l1: 0.0386845
[2399]	valid_0's l2: 0.00288036	valid_0's l1: 0.0386836
[2400]	valid_0's l2: 0.00288022	valid_0's l1: 0.0386825
[2401]	valid_0's l2: 0.00288016	valid_0's l1: 0.0386821
[2402]	valid_0's l2: 0.00288015	valid_0's l1: 0.0386818
[2403]	valid_0's l2: 0.0028801	valid_0's l1: 0.0386813
[2404]	valid_0's l2: 0.00288009	valid_0's l1: 0.0386812
[2405]	valid_0's l2: 0.00288005	valid_0's l1: 0.0386807
[2406]	valid_0's l2: 0.00288002	valid_0's l1: 0.0386804
[2407]	valid_0's l2: 0.00287996	valid_0's l1: 0.0386796
[2408]	valid_0's l2: 0.00287984	valid_0's l1: 0.0386783
[2409]	valid_0's l2: 0.00287978	valid_0's l1: 0.038678
[2410]	valid_0's l2: 0.00287973	valid_0's l1: 0.0386774
[2411]	valid_0's l2: 0.00287967	valid_0's l1: 0.0386763
[2412]	valid_0's l2: 0.00287962	valid_0's l1: 0.0386756
[2413]	valid_0's l2: 0.00287952	valid_0's l1: 0.03

[2546]	valid_0's l2: 0.00286963	valid_0's l1: 0.0385736
[2547]	valid_0's l2: 0.00286947	valid_0's l1: 0.0385723
[2548]	valid_0's l2: 0.00286945	valid_0's l1: 0.0385718
[2549]	valid_0's l2: 0.00286934	valid_0's l1: 0.0385708
[2550]	valid_0's l2: 0.00286928	valid_0's l1: 0.03857
[2551]	valid_0's l2: 0.00286924	valid_0's l1: 0.0385695
[2552]	valid_0's l2: 0.00286915	valid_0's l1: 0.0385686
[2553]	valid_0's l2: 0.00286905	valid_0's l1: 0.0385677
[2554]	valid_0's l2: 0.00286894	valid_0's l1: 0.0385665
[2555]	valid_0's l2: 0.00286891	valid_0's l1: 0.0385658
[2556]	valid_0's l2: 0.00286888	valid_0's l1: 0.0385654
[2557]	valid_0's l2: 0.00286887	valid_0's l1: 0.038565
[2558]	valid_0's l2: 0.00286884	valid_0's l1: 0.0385647
[2559]	valid_0's l2: 0.00286878	valid_0's l1: 0.0385642
[2560]	valid_0's l2: 0.00286874	valid_0's l1: 0.0385636
[2561]	valid_0's l2: 0.00286869	valid_0's l1: 0.038563
[2562]	valid_0's l2: 0.00286862	valid_0's l1: 0.0385624
[2563]	valid_0's l2: 0.00286858	valid_0's l1: 0.0385

[2697]	valid_0's l2: 0.00285953	valid_0's l1: 0.0384655
[2698]	valid_0's l2: 0.0028595	valid_0's l1: 0.0384647
[2699]	valid_0's l2: 0.00285942	valid_0's l1: 0.038464
[2700]	valid_0's l2: 0.00285936	valid_0's l1: 0.0384636
[2701]	valid_0's l2: 0.00285934	valid_0's l1: 0.0384633
[2702]	valid_0's l2: 0.00285924	valid_0's l1: 0.0384624
[2703]	valid_0's l2: 0.00285917	valid_0's l1: 0.0384618
[2704]	valid_0's l2: 0.00285911	valid_0's l1: 0.0384612
[2705]	valid_0's l2: 0.00285905	valid_0's l1: 0.0384605
[2706]	valid_0's l2: 0.00285896	valid_0's l1: 0.0384596
[2707]	valid_0's l2: 0.00285885	valid_0's l1: 0.0384586
[2708]	valid_0's l2: 0.00285878	valid_0's l1: 0.038458
[2709]	valid_0's l2: 0.00285872	valid_0's l1: 0.0384571
[2710]	valid_0's l2: 0.00285862	valid_0's l1: 0.0384559
[2711]	valid_0's l2: 0.00285859	valid_0's l1: 0.0384556
[2712]	valid_0's l2: 0.00285858	valid_0's l1: 0.0384546
[2713]	valid_0's l2: 0.00285853	valid_0's l1: 0.038454
[2714]	valid_0's l2: 0.00285845	valid_0's l1: 0.0384

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.4,
       learning_rate=0.025, max_depth=9, min_child_samples=110,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=2800,
       n_jobs=8, num_leaves=120, objective='regression', random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=0.95,
       subsample_for_bin=200000, subsample_freq=0, verbose=1)

In [31]:
ypred = gbm.predict(test_X_A)
print_test_result(ypred, test_y_A)

预测误差在百分之 3以内数目为：37021,所占比例为：25.1174%
预测误差在百分之 5以内数目为：58260,所占比例为：39.5272%
预测误差在百分之 8以内数目为：83305,所占比例为：56.5193%
预测误差在百分之10以内数目为：95924,所占比例为：65.0809%


In [32]:
by_model = test_A[['model_code', 'count', 'residual']]
by_model['pre_residual'] = ypred
by_model['pre_err'] = abs(by_model['residual'] - by_model['pre_residual'])
by_model['percent_err'] = by_model['pre_err']/by_model['residual']

data_result = pd.DataFrame(columns={'model_code':str, 'median_res':float})
data_result['model_code'] = list(by_model.groupby(['model_code'])['percent_err'].quantile(0.5).index)
data_result['median_res'] = list(by_model.groupby(['model_code'])['percent_err'].quantile(0.5))

print(data_result[data_result['median_res']<=0.03].shape[0], 
      data_result[data_result['median_res']<=0.03].shape[0]/data_result.shape[0])
print(data_result[data_result['median_res']<=0.05].shape[0], 
      data_result[data_result['median_res']<=0.05].shape[0]/data_result.shape[0])
print(data_result[data_result['median_res']<=0.08].shape[0], 
      data_result[data_result['median_res']<=0.08].shape[0]/data_result.shape[0])
print(data_result[data_result['median_res']<=0.1].shape[0], 
      data_result[data_result['median_res']<=0.1].shape[0]/data_result.shape[0])

1791 0.1331103678929766
3857 0.28665923448532143
6972 0.5181716833890747
8412 0.6251950947603121


In [33]:
data_result = data_result[['model_code', 'median_res']]
data_result.to_csv(curr_dir+"2020-11-02采购数据-A级车-误差中位数.csv", header=True, index=False)